# Рекомендация тарифов <a id="intro"></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание" data-toc-modified-id="Описание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание</a></span></li><li><span><a href="#Импорт-необходимых-библиотек" data-toc-modified-id="Импорт-необходимых-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Импорт необходимых библиотек</a></span></li><li><span><a href="#Разбивка-данных-на-выборки" data-toc-modified-id="Разбивка-данных-на-выборки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Разбивка данных на выборки</a></span></li><li><span><a href="#Настройка,-обучение-и-проверка-моделей-на-валидационной-выборке" data-toc-modified-id="Настройка,-обучение-и-проверка-моделей-на-валидационной-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Настройка, обучение и проверка моделей на валидационной выборке</a></span></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span></li><li><span><a href="#Проверка-моделей-на-вменяемость" data-toc-modified-id="Проверка-моделей-на-вменяемость-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверка моделей на вменяемость</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Описание

Оператор мобильной связи «Мегалайн» установил, что многие клиенты пользуются архивными тарифами и планирует построить систему, способную анализировать поведение клиентов и предлагать пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель с максимально большим значением Accuracy для задачи классификации, которая выберет подходящий тариф. Доля правильных ответов должна быть не менее 0.75.

## Импорт необходимых библиотек

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Разбивка данных на выборки

Прочитаем информацию из файла.

In [2]:
data = pd.read_csv("users_behavior.csv")

Разобьем датафрейм на целевой признак и все остальные признаки.

In [3]:
features = data.drop("is_ultra", axis=1)
target = data["is_ultra"]

Разобъем целевой признак и все остальные признаки на обучающую, валидационную, тестовую выборки в сотношении 3:1:1 

In [4]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)
print(features_train.shape) #  признаки обучающей выборки
print(target_train.shape) # целевой признак обучающей выборки
print(features_valid.shape) #  признаки валидационной выборки
print(target_valid.shape) # целевой признак валидационной выборки
print(features_test.shape) # признаки тестовой выборки
print(target_test.shape) # целевой признак тестовой выборки

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


[В начало](#intro)

## Настройка, обучение и проверка моделей на валидационной выборке

Используем модель дерево решений с гиперпараметром глубины для получения наилучшего Accuracy.

In [5]:
# Дерево решений
best_tree_model = None
best_tree_valid_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)
    result = accuracy_score(prediction_valid, target_valid)
    if result > best_tree_valid_result:
        best_tree_valid_result = result
        best_tree_model = model
        best_depth = depth
print("Accuracy налучшей модели на валидационной выборке:", best_tree_valid_result, "Глубина модели:", best_depth)    

Accuracy налучшей модели на валидационной выборке: 0.7651632970451011 Глубина модели: 3


Используем модель случайного леса. Настроим для начала гиперпараметр количество оценщиков, которая дает наилучший результат Accuracy.

In [6]:
# Модель случайный лес без глубины
best_forest_model = None
best_forest_valid_result = 0
best_est = 0
for est in range(1, 15):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, prediction_valid)
    if result > best_forest_valid_result:
        best_forest_valid_result = result
        best_forest_model = model
        best_est = est
print("Accuracy наилучшей модели на валидационной выборке", best_forest_valid_result, "Количество оценщиков:", best_est)

Accuracy наилучшей модели на валидационной выборке 0.7884914463452566 Количество оценщиков: 10


Используем гиперпараметры количества оценщиков и глубины модели одновременно.

In [7]:
# Модель случайный лес с глубиной
best_forest_model = None
best_forest_valid_result = 0
best_depth = 0
best_est = 0
for est in range(10, 61, 10):
    for depth in range (1, 12):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        prediction_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, prediction_valid)
        if result > best_forest_valid_result:
            best_forest_valid_result = result
            best_forest_model = model
            best_est = est
            best_depth = depth
print("Accuracy налучшей модели на валидационной выборке:", best_forest_valid_result, "Глубина модели:", best_depth, "Количество оценщиков:", best_est)   

Accuracy налучшей модели на валидационной выборке: 0.7978227060653188 Глубина модели: 10 Количество оценщиков: 50


Модель случайного леса с настройками гиперпараметров количества оценщиков и глубины дает результат лучше модели, не использующий гиперпараметр глубины.

Обучим логистическую регрессию.

In [8]:
# Логистическая регрессия
logistic_model = LogisticRegression(random_state=12345, solver="lbfgs")
logistic_model.fit(features_train, target_train)
prediction_valid = logistic_model.predict(features_valid)
logistic_valid_result = accuracy_score(target_valid, prediction_valid)
print("Accuracy модели:", logistic_valid_result)

Accuracy модели: 0.7262830482115086


Выведем результаты всех моделей.

In [9]:
print("Accuracy моделей на валидационной выборке:")
print("Accuracy модели дерева решений:", best_tree_valid_result)
print("Accuracy модели случайный лес:", best_forest_valid_result)
print("Accuracy модели логистическая регрессия:", logistic_valid_result)

Accuracy моделей на валидационной выборке:
Accuracy модели дерева решений: 0.7651632970451011
Accuracy модели случайный лес: 0.7978227060653188
Accuracy модели логистическая регрессия: 0.7262830482115086


[В начало](#intro)

##  Проверка моделей на тестовой выборке

In [10]:
prediction_tree_test = best_tree_model.predict(features_test)
prediction_forest_test = best_forest_model.predict(features_test)
prediction_logistic_test = logistic_model.predict(features_test)

tree_test_result = accuracy_score(target_test, prediction_tree_test)
forest_test_result = accuracy_score(target_test, prediction_forest_test)
logistic_test_result = accuracy_score(target_test, prediction_logistic_test)

print("Accuracy моделей на тестовой выборке:")
print("Accuracy модели дерева решений:", tree_test_result)
print("Accuracy модели случайный лес:", forest_test_result)
print("Accuracy модели логистическая регрессия:", logistic_test_result)

Accuracy моделей на тестовой выборке:
Accuracy модели дерева решений: 0.7869362363919129
Accuracy модели случайный лес: 0.7993779160186625
Accuracy модели логистическая регрессия: 0.7589424572317263


[В начало](#intro)

##  Проверка моделей на вменяемость

Проверим модель на вменяемость при помощи модели DummyClassifier и доли тарифов "Smart" и "Ultra".

In [11]:
dummy_model = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_model.fit(features_train, target_train)
dummy_model_result = dummy_model.score(features_test, target_test)
print("Accuracy DummyClassifier:", dummy_model_result)

Accuracy DummyClassifier: 0.6951788491446346


У нашей модели Accuracy выше, поскольку она не предсказывает просто самое частое значение в классах, как это делает модель DummyClassifier с гиперпараметром strategy="most_frequent". Простое предсказывание самого популярного класса дает ниже результат, наша модель вменяема. 

Проверим соотношение пользователей тарифов "Smart" и "Ultra", если модель адекватна ее Accuracy будет выше. 

In [12]:
(data["is_ultra"]==0).sum() / data.shape[0]

0.693528313627878

В обоих случая Accuracy нашей модели выше - успех!

## Выводы

Все представленные модели является вменяемы и могут быть применимы в различных ситуациях в зависимости от бизнес-потребности и особенностей каждой модели.

Самая быстрая модель со средним качеством является логистическая регрессия, ее можно использовать, если требуется быстрое предсказание на лету по входным параметрам, где можно закрыть глаза на небольшую неточность модели, скорость его основное преимущество.

Если не треубется предсказывать на лету, рекомендуется использовать более точную модель - модель случайного леса. Такие предсказания можно проводиться по крону, который срабатывает по расписанию.

В нашем случае Accuracy должно быть не меньше 0.75, на тестовой выборке такой результат выдали модели дерева решений и случайный лес.

[В начало](#intro)